In [1]:
from utils import *
from CNN import *
from random import *
import copy
import numpy as np
import cv2
import os

# Load data

In [2]:
# load image and label from folder
def load_images_from_folder(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_path = os.path.join(folder, label)
        if os.path.isdir(label_path):
            for filename in os.listdir(label_path):
                img_path = os.path.join(label_path, filename)
                if os.path.isfile(img_path):
                    image = cv2.imread(img_path)
                    if image is not None:
                        resized_image = cv2.resize(image, (64, 64), interpolation=cv2.INTER_AREA)
                        images.append(resized_image)
                        labels.append(label)  
    return images, labels

In [3]:
trainDir = '../data/archive/PandasBears/Train/'
testDir = '../data/archive/PandasBears/Test/'

train_x, train_y = load_images_from_folder(trainDir)
test_x, test_y = load_images_from_folder(trainDir)

In [4]:
len(train_x[0])

64

In [5]:
train_y_encoded = [1 if item == 'Bears' else 0 for item in train_y]

# Ncross Val

In [6]:
def n_fold_cross_validation(data, labels, n_folds):
    if len(data) != len(labels):
        print("data and label is not the same length")
        return 
    fold_data = []
    data_labels = list(zip(data, labels))
    np.random.shuffle(data_labels)

    fold_size = len(data) // n_folds

    for i in range(n_folds):
        start = i * fold_size
        end = (i + 1) * fold_size if i < n_folds - 1 else len(data)
        
        test_set = data_labels[start:end]
        train_set = data_labels[:start] + data_labels[end:]
        
        fold_data.append((train_set, test_set))

    return fold_data

In [7]:
n_folds = 10

folds = n_fold_cross_validation(train_x, train_y_encoded, n_folds)

In [8]:
len(folds)

10

In [9]:
len(folds[0])

2

In [10]:
len(folds[0][0])

450

In [11]:
folds[0][0][0][1]

1

In [12]:

folds[0][0][0][0].shape

(64, 64, 3)

In [13]:
# train_set array of 

In [14]:
for i, (train_set, test_set) in enumerate(folds):
    print(train_set[0][0])
    print(f"Fold {i+1} - Train Set Length: {len(train_set)}, Test Set Length: {len(test_set)}")

[[[ 99 184 174]
  [102 184 172]
  [103 192 180]
  ...
  [ 82 169 158]
  [ 84 171 161]
  [ 87 175 161]]

 [[100 180 168]
  [103 184 168]
  [106 194 177]
  ...
  [ 83 173 163]
  [ 85 174 163]
  [ 88 178 163]]

 [[103 176 164]
  [103 182 164]
  [107 193 172]
  ...
  [ 90 177 164]
  [ 89 180 164]
  [ 90 184 163]]

 ...

 [[ 50  29  13]
  [ 46  21   7]
  [ 46  21   7]
  ...
  [ 22   8   7]
  [ 26  15  16]
  [ 18  12  12]]

 [[ 42  18   5]
  [ 45  20   6]
  [ 46  20   8]
  ...
  [ 20   7   5]
  [ 29  19  17]
  [ 25  17  15]]

 [[ 43  17   4]
  [ 39  14   3]
  [ 46  20   8]
  ...
  [ 17   5   3]
  [ 36  26  22]
  [ 27  18  14]]]
Fold 1 - Train Set Length: 450, Test Set Length: 50
[[[ 57 142 113]
  [ 52 141 108]
  [ 51 139 106]
  ...
  [ 69 135 109]
  [ 69 136 109]
  [ 71 136 110]]

 [[ 62 149 115]
  [ 56 147 110]
  [ 56 146 110]
  ...
  [ 70 138 111]
  [ 70 140 113]
  [ 73 140 113]]

 [[ 69 155 120]
  [ 60 152 114]
  [ 57 152 113]
  ...
  [ 73 140 113]
  [ 73 141 114]
  [ 75 141 115]]

 ...



In [15]:
def evaluate(label, prediction):
    if len(label) != len(prediction):
        print("Label and prediction have differenct length")
        return

    correct = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    true_negative = 0

    for i in range(len(label)):
        if label[i] == prediction[i]:
            correct += 1
            if label[i] == 1:
                true_positives += 1
            else:
                true_negative +=0
        else:
            if label[i] == 1:
                false_negatives += 1
            else:
                false_positives += 1

    accuracy = correct / len(label)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score,
        "confusion_matrix" : [true_positives, false_positives, false_negatives, true_negative]
    }

    return metrics


In [16]:
model_list = []
metric_list = []
for i, (train_set, test_set) in enumerate(folds):
    print("Model", i)
    model = CNN(input_shape_=(3, 64, 64))
    print(folds[0][0][0][0].shape)
    model.add(ConvolutionLayer(8, [3,3], 0, (1,1)))
    model.add(DetectorLayer())
    model.add(PoolingLayer((2,2), "max"))
    model.add(FlattenLayer())
    model.add(DenseLayer(8, "relu"))

    model.add(DenseLayer(1, "sigmoid"))
    model.compile()
    #   
    #  train_label= [(image,label), .....]
    data = []
    label = []
    
    for j in train_set[:50]:
        
        # print(j[0])
        red = separateChannel(j[0], 0)
        green = separateChannel(j[0], 1)
        blue = separateChannel(j[0], 2)
        split_matrix = np.array([red,green,blue])
        data.append(split_matrix)
        label.append(j[1])
    print("training ", i)
    # print(data.shape)
    model.fit(np.array(data), label, 2)
    
    data_test = []
    label_test = []

    for j in test_set[:5]:
        
        # print(j[0])
        red = separateChannel(j[0], 0)
        green = separateChannel(j[0], 1)
        blue = separateChannel(j[0], 2)
        split_matrix = np.array([red,green,blue])
        data_test.append(split_matrix)
        label_test.append(j[1])
        
    metric = evaluate(model.predict(),label )
    print("Metric", i, metric)
    metric_list.append(metric)
    model_list.append(model)
    

Model 0
(64, 64, 3)
training  0


c:\Users\rongg\kuliah\Semester 9\MLL\TugasBesar_IF4074PembelajaranMesinLanjut\Src\utils.py:80: RuntimeWarning: invalid value encountered in double_scalars
  result[i][j] = arr1[i] * arr2[j]




Epoch  1
Accuracy: 


In [ ]:
data_test = []
label_test = []

for j in test_set[:5]:
    
    # print(j[0])
    red = separateChannel(j[0], 0)
    green = separateChannel(j[0], 1)
    blue = separateChannel(j[0], 2)
    split_matrix = np.array([red,green,blue])
    data_test.append(split_matrix)
    label_test.append(j[1])
    
evaluate(model.predict(),label )